# Projeto 2

In [1]:
import moviepy.editor as mpe
import numpy as np
import matplotlib.pyplot as plt
import math

#### Carrega video original

In [2]:
original = mpe.VideoFileClip("inicial.mp4")

#### Iverter a imagem, no eixo, a cada 20 segundos

In [3]:
# Inverte um frame
def refl_frame(frame):
    frame_refl = np.zeros(shape=frame.shape, dtype=np.uint8)
    frame_refl[::1, ::-1] = frame

    return frame_refl

# Dentro de um intervalo de 40s 
# nos 20 primeiros o video se mantem como no orginal 
# e nos 20 ultimos a imagem é invertida
def refl_frame_each_20(get_frame, t):
    time = math.floor(t)
    if (time % 40) in range(20, 40):
        return refl_frame(get_frame(t))
    return get_frame(t)

video_mod: mpe.VideoClip = original.fl(refl_frame_each_20)

#### Diminuir o som, de forma gradativa, a cada 30 segundos, de forma que os últimos 10 segundos fique sem silêncio (sem som)

In [4]:
total_duration = video_mod.duration
total_parts = math.floor((total_duration) / 30)

def calc_factors_array(t, audio_shape):
    end_mask = t >= total_duration - 10
    intervals = t // 30
    
    factors = 1 - intervals / total_parts
    factors[end_mask] = 0

    return np.tile(np.maximum(factors, 0), (audio_shape[1], 1)).T

def reduce_volume(get_audio_frame, t):
    af = get_audio_frame(t)
    factors = calc_factors_array(t, af.shape)

    return af * factors


audio_mod = video_mod.audio.fl(reduce_volume)
video_mod = video_mod.set_audio(audio_mod)

#### Após o primeiro minuto, 60 segundos, fazer um corte para o segundo 80

In [5]:
video_mod_start = video_mod.subclip(0, 60)
video_mod_cut = video_mod.subclip(60, 80)
video_mod_end = video_mod.subclip(80)

video_mod = mpe.concatenate_videoclips([video_mod_start, video_mod_end])

#### Por fim, insira os 20 segundos cortados acima no fim do video

In [6]:
final = mpe.concatenate_videoclips([video_mod, video_mod_cut])

#### Salvar Video final

In [7]:
final.write_videofile("final.mp4")

Moviepy - Building video final.mp4.
MoviePy - Writing audio in finalTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video final.mp4



Moviepy - Done !
Moviepy - video ready final.mp4
